In [ ]:
import pandas as pd
from pyprojroot import here
import numpy as np

from nutils import interval_score

In [ ]:
TARGET = 'occ'

In [ ]:
true_matrix = pd.read_csv(
    here() / f'data/processed/true_matrices/{TARGET}.csv',
    index_col='datetime',
    parse_dates=True
)

In [ ]:
result_list = list()

for lpath in here('data/processed/prediction_matrices/05').glob('*.csv'):
    
    result = dict()
    
    parts = lpath.stem.split('-')
    
    result['Model'] = parts[1].upper()
    result['FS'] = parts[2].upper()
    result['HPO'] = parts[3]
    
    l = pd.read_csv(
        lpath, 
        index_col='datetime', 
        parse_dates=True
    )
    
    upath = here() / 'data/processed/prediction_matrices/95'
    upath = upath / f"{lpath.stem}.csv"
    
    u = pd.read_csv(
        upath, 
        index_col='datetime', 
        parse_dates=True
    ).dropna()
    print(lpath)
    true_vector = true_matrix.loc[u.index].dropna()
    
    l_vector = l.loc[true_vector.index].values.flatten()
    u_vector = u.loc[true_vector.index].values.flatten()
    true_vector = true_vector.values.flatten()
    
    try:
        msis = interval_score(
            observations=true_vector,
            alpha=0.05, 
            q_left=l_vector, 
            q_right=u_vector,
            mean=True,
            scaled=True,
            seasonality=24
        )
    except Exception as e:
        msis = np.nan
        print(f'Encountered error when calculating MSIS for {p.stem}')
        print(e)
        
    result['MSIS'] = msis.round(2)
    result_list.append(result)

In [ ]:
df = pd.DataFrame(result_list)
df = df.sort_values(by='MSIS', ascending=False)
df = df.set_index('Model')

In [ ]:
df

In [ ]:
df.to_latex(
    buf=here() / 'output/tables/msis.tex',
    float_format='%.2f',
    column_format='rccc',
    label='tab:msis',
    caption='Mean scaled interval score (MSIS) for the tested models',
    position='h'
)